# **Generator explanation**

<!-- **Why I wanted to do this project**

Before I learned to program / decided I wanted to learn to do data science, I ran user-testing programs for hardware products at a tech company. There are a lot of people getting into data science, and while I still feel like a beginner at programming, I'm pretty confident in my business background. So I wanted a project to show that. When I first started, this project had almost nothing to do with business, but it just kind of bled through - the connections were already there.

- so maybe it's better to just say that? think about the message and try to get it clear.
- I also like this stuff anyway - ask me about the case study I wrote for a business fraternity in 2018. (I picked this restaurant because I thought the name was funny - I was going to use another restaurant I also wrote a case about called the Himalayan Chimney, but they changed their website to just an ordering screen because of covid, which wasn't as fun).

**Showcasing A/B testing is hard**

Anyway, it's hard to showcase A/B testing because you need a product and users. And every company seems to want A/B testing experience. So instead of giving up, I found a sneaky way around it. When a company does A/B testing, they just look at the data to see how users interact with a change. So instead of actually implementing a change and looking at the data, we can just regenerate data for a month and pretend that's the sales data from the users. It's just order data, so it's not as good as real-time user data (like click-through-rates or stuff like that), but it's not likely most restaurants will have that kind of data either - it's a nice, simple example.

- also gives us a chance to talk about the difference between a/b testing and other types of user testing
- don't forget about the pitfalls of A/B testing! How does that apply to this case?

**Notebooks are independent** (!) will probably move to readme

In each workbook, we assume we have no knowledge of other notebooks.

A truer chronological workflow might look like this:

1. Get data
    1. Write generator, generate data [data_generator.py]
    2. Document generator design [generator_explanation.ipynb]
    
    
2. Explore data
    1. Explore, validate data (suggest business recs / potential tests) [data_analysis.ipynb]
    
    
3. Design test
    1. Design experiment for accepted tests [experiment_analysis.ipynb]
    
    
4. Implement test
    1. Regenerate data for test [data_generator.py]
    2. Document re-generator design [generator_explanation.ipynb]
    
    
5. Evaluate findings
    1. Analyze experiment, explain findings [experiment_analysis.ipynb] -->

This notebook covers the design choices for the data generator. **Don't look if you don't want spoilers on the dataset!**

### Table of Contents: 
- Data generator overview
- Two key questions (market sizing, order sizing)
- Improvements & other notes
<!-- - Evaluate data -->

# Data generator overview


The generator creates order data for a real Thai restaurant by estimating the number of customers and what they are likely to order. Its expected output is a monthly .csv file per specified month/year.

The number of customers is based on the restaurant's maximum possible output multiplied by Google's popular times occupancy hourly estimation. What the customers order is based on weights I assigned to the menu categories and items (I picked based on what I thought customers in the area might really order, and checked with some friends so hopefully it's not too biased). These factors both have separate random multipliers and fixed monthly and yearly weights.

I'll quickly overview the columns and their distributions and then talk more about how I made my assumptions.

<!-- having core assumptions here might be a better format - check how research papers are outlined... -->

### Data fields
- **Order ID**: Unique order ID (orders with more than one item are a second line with the same ID).
        Number of orders per hour is based on estimated occupancy from popular times. There's some randomness too. And some annual/seasonal values. We'll explore this one further below.
    
    
- **Item**: Menu item ordered. See menu for possible values.
        I assigned weights to each menu category and item. The generator randomly selects a category, then an item from that category. 


- **Quantity Ordered**: Amount of the menu item ordered.
    Follows exponential distribution. There's a chance to order an additional item.

- **Price Each**: Price per item ordered, same as listed on menu.


- **Order Date**: Order date and time in YYYY-MM-DD hh:mm:ss format.

# Core assumptions

There are two main factors that affect how much revenue the restaurant generates:
1. How many orders the restaurant gets per month (external)
2. How much people spend per order (internal)

<!-- Each of these factors  -->

Here's a revenue model to help visualize the process. The two main things that affect sales are the number of sales the restaurant gets, and how much each sale is worth. 

### Revenue model: two key questions

![](..\thaitanic/revenue_model.jpg)

Pictured is a revenue model to help us understand the key drivers of sales. The two key factors are how many customers the restaurant gets, and what they order. Focusing on the factors that affect those drivers helps us create more realistic data.

<!-- link to [slides](https://docs.google.com/presentation/d/1PKQtxfG-HTdD3Nb0Tvh_jYqEWOlPUWJRwaHT3MVXpoM/edit?usp=sharing)  -->

# 1. How many orders does the restaurant get per month?

### Estimating market size


We want to find out the maximum amount of orders the restaurant might get per month. We can get this by multiplying the restaurant's max capacity by the number of orders the resturant can handle, which is the average meal length. 

##### Restaurant capacity

Based on photos from 
[Yelp](https://www.yelp.com/biz_photos/thaitanic-streetfood-sausalito-2)
and 
[Google](https://www.google.com/maps/uv?pb=!1s0x80858451b4f18a91:0xa561eb4585721e9d!3m1!7e115!4s)
(can't rehost images due to copyright), it looks like there are **30 seats total** (20 indoor, 10 outdoor).

##### # of servable meals + takeout

Google indicates the average time spent per meal is 15 minutes to an hour. One hour seems high for an average, and 30 minutes seems a bit low, but since they do takeout, I'm okay with using 30 minutes. If we wanted to estimate for takeout more accurately, we might use kitchen output instead of physical capacity but that seems much more complicated for a small improvement that would be tough to measure too.

The restaurant is open for 8 hours a day, so we're saying they can serve at max 16 meals per seat per day. At 30 seats, we estimate that they can serve (30 * 16 =) **480 meals per day**. This sounds high, but it's really the amount of meals the restaurant would be able to fulfill if they had lines out the door from the minute they open to the minute they close. 

In the actual data, we probably wouldn't want to see any numbers above 400 except for maybe the busiest hours of the busiest days of the busiest months.

<!-- For reference...? -->

### Estimating actual demand

Google also provides "Popular Times" estimates. Here are the Wednesday (least busy day) and Saturday (most busy day).

**Wednesday popular times**

![](..\thaitanic/data/popular_times/wednesday.PNG) 

**Saturday Popular times**

![](..\thaitanic/data/popular_times/saturday.PNG)

The difference between days in this data is HUGE - the busiest day is almost three times as busy as the least busy day. It's hard to make concrete assumptions because Google doesn't provide a scale for the data - based on this, you would assume the restaurant is empty for most hours of most weekdays. I don't think that's totally true, so I assumed they were relative values. 

I interpreted the bottom line to be 50% occupancy, and each additional line to be 25% occupancy. This means an hour with over two bars of demand is over 100% occupancy, which would just be represented as a wait time. I then divided this value by 100 to turn it into a multiplier, and assigned one of these multipliers to every hour of every weekday. My method can be represented by this formula:

$f(x) = \frac{25x + 50}{100}$

The values I pulled are specific to 2020 so there's a chance they're uncharacteristically low, but I checked against restaurants in areas not heavily affected by covid and they seemed similar enough to just keep.

Note that if we didn't have Google's data, we could just estimate by the hour, or generate data on a per-day basis (which I'm glad we didn't do because that would be much too uniform). We might also be able to increase accuracy by evaluating the local population, but at that point I would rather sit outside with a clipboard.

##### Annual, seasonal weights

I also added weights to decrease the number of orders. Annually, I just wanted the restaurant to be trending upwards. I checked some google market research for this.

I wanted the holiday months to be really high, and the summer months to be slightly higher. The rest ofthe months could be whatever, I put them at 80 (so at 50% occupancy all day - zero bars - we're now looking at about 190 orders. It works out pretty well to start from a high amount and use decreasing multipliers.

This helps us size the total market. The idea is that we can estimate the total realistic output from the restaurant, and then apply modifiers to decrease from there. 

### Compensating for quantity ordered distribution

Using an exponential distribution in quantity ordered increases our total quantity ordered by about 66%, so we need a negative modifier to compensate for that. I feel comfortable that our number of orders accurately represents the number of orders per seat.

I used 0.7 as a modifer insted of 0.6 (1/1.66) because there still are some guests who order more than one dish per seat. Seems reasonable since no guests who order less than one dish per seat!

# 2. What do people spend on orders?

I peeked at yelp too for input and considered scraping them for ratings, but their robot.txt disallows it.

This part was really easy... I just added in the weights from the menu. 

### Menu weights

I copied the menu and added weights per category and then per item. The category weights were basd on whether or not I thought someone would want an item from there if they actually ordered. It also had to do with how good the items were. 

Then for the items in the category, there are relative weights that determine which item should be preferred once the category is selected.

I peeked at the local population and went with my gut on what I thought would actually be ordered (it hurt me to rank Pad Thai above Pad See Ew). I also asked some friends for input on this. I initially considered asking a bunch of people on the internet to tell me what they would actually order, but I figured it was overkill (might be fun for another project though).

### Order quantity 

I kind of just added this in at the end, but it felt like the values were really low so I threw in a chance to order more of an item.

And also a chance to order a second item. Both of these increase total quantity ordered and therefore sales, but I think it's fine.

### Adding a second item to an order

There's a small chance to order a second item. It's really high if it's a dessert, and it's kind of high if there's an appetizer. I actually forgot about the appetizer part, then rediscovered it in the analysis.

# Improvements

- Order IDs don't match Order Date. They're all within the hour, but they're random. It doesn't really make sense that the first order would be 10 minutes from closing, while the next order after that is halfway through the shift. The solution is probably to generate from a minute basis, but it seems like a small enough detail to skip for now. **I think this is the only real flaw in the data** - the rest are just minor inaccuracies.
- Code is a bit slow. There are some list comprehensions in the loop that aren't very elegant because I was accessing multi-element lists in a dictionary. It felt like too much to explicitly get lists of the weight element from the dictionary, so I put it in the loop. I'm sure there's also a cleaner way I could've added options for additional orders, but will revisit when I have a deeper understanding of programming.
- I think my usage of nested loops is appropriate here, but wow it's ugly to see that many nested loops. I'm sure I'll realize a better way to have built that in the future.
- This applies to all of the notebooks in this project, but I think they get messy when there's too much content, and I'm a bit verbose. I'll work on it.
- I used an exponential distribution on quantity ordered which isn't wrong, but it feels too uniformly exponential. I would imagine most orders to be one, then two through four, then some ives and sixes, then almost nothing, then a bunch of orders from corporate catering around 10. Instead, it follows a smooth exponential curve which is a bit too pretty.
- I calculated the orders based on a single person, and changed the quantity ordered to an exponential distribution without checking how that affects the total quantity ordered. The data is a bit conservative, but the orders and therefore sales are bit higher than I intended. But it's a family style place so it's not too bad. 

If you're reading this and have advice for any of the above (or any feedback on the project at all, really!) please let me know. Pull requests or emails work.

# Other notes

I did 2017 - 2019 data because I didn't want to touch covid in this dataset. The effects are complicated enough that mocking data doesn't seem to be the best fit (if you really wanted to, you could use machine learning to emulate datasets instead).

You might be able to make covid data by setting the weight to a random (low) value that can sometimes be zero. But it feels out of the scope of this problem.

# Random state

I didn't use a random state because random.choices SPECIFICALLY does not work with np.random.RandomState(). random.choices worked fine. It looks like it was added to the random library but isn't supported by numpy yet. 